In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import json
import requests
from sklearn.preprocessing import MinMaxScaler

## Loading in the dataset and collecting random samples from it
The dataset contains over 30 million transactions.

We'll randomly sample 10,000 non-fraudulent transactions and combine them with the fraudulent transactions

In [ ]:
bitcoin_txs = pd.DataFrame()
fraud_txs = pd.DataFrame()

# Load the dataset in chunks
for chunk in pd.read_csv('/content/drive/MyDrive/final year/DG_out.csv', chunksize=100000):

    # Filter rows a transaction has any malicious inputs, outputs or itself is malicious
    malicious_rows = chunk[(chunk['all_malicious']==1)]

    # Concatenate the malicious rows to the DataFrame
    fraud_txs = pd.concat([fraud_txs, malicious_rows])

    # Take a random sample from each chunk making sure they are non-fraudulent
    data_samples = chunk[(chunk['in_malicious']==0) & (chunk['out_malicious']==0) & (chunk['is_malicious']==0) & (chunk['out_and_tx_malicious']==0) & (chunk['all_malicious']==0)].sample(n=33, random_state=42)

    # Add each sample
    bitcoin_txs = pd.concat([bitcoin_txs, data_samples])

# Concat to a single dataset
bitcoin_txs = pd.concat([bitcoin_txs, fraud_txs])

# Write file
bitcoin_txs.to_csv('/content/drive/MyDrive/final year/ml/data/bitcoin_txs.csv',index=False)

## Get info from Blockchain.com

In [ ]:
metadata_list = []
result_df = pd.DataFrame()

# Assuming 'malicious' is a DataFrame with a 'tx_hash' column
for txhash in sample_df['tx_hash']:
    # Specify the URL you want to request
    url = "https://blockchain.info/rawtx/" + txhash

    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        count+=1
        # Normalize the JSON response to a dataframe and add to the list
        metadata_df = pd.json_normalize(response.json())
        metadata_list.append(metadata_df)
        print('API CALL: ' + str(count))

    else:
        # Print an error message if the request was not successful
        print(f"Error: {response.status_code} - {response.text}")


# Concatenate list of dataframes to a single one and save it to a file
result_df = pd.concat(metadata_list, ignore_index=True).to_csv('/content/drive/MyDrive/final year/ml/data/blockchaincom_data.csv')


# concat into a single dataset

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/final year/btc_tx_anomaly_detection/data/bitcoin_txs.csv', index_col=False)
df2 = pd.read_csv('/content/drive/MyDrive/final year/btc_tx_anomaly_detection/data/blockchaincom_data.csv', index_col=False)

In [ ]:
# Merge DataFrames based on the common column 'hash' in df1 and 'x_hash' in df2
merged_df = pd.merge(df1, df2, left_on='tx_hash', right_on='hash')

In [ ]:
merged_df.keys()

Index(['tx_hash', 'indegree', 'outdegree', 'in_btc', 'out_btc', 'total_btc',
       'mean_in_btc', 'mean_out_btc', 'in_malicious', 'out_malicious',
       'is_malicious', 'out_and_tx_malicious', 'all_malicious', 'Unnamed: 0',
       'hash', 'ver', 'vin_sz', 'vout_sz', 'size', 'weight', 'fee',
       'relayed_by', 'lock_time', 'tx_index', 'double_spend', 'time',
       'block_index', 'block_height', 'inputs', 'out'],
      dtype='object')

# Data cleaning

In [ ]:
merged_df = merged_df.rename(columns={'out_and_tx_malicious':'is_fraud'})

In [ ]:
# remove all uncessary columns
merged_df = merged_df.drop(['tx_hash',
                           'indegree',
                           'outdegree',
                           'in_malicious',
                           'out_malicious',
                           'is_malicious',
                           'all_malicious',
                           'Unnamed: 0',
                           'hash',
                           'ver',
                           'weight',
                           'relayed_by',
                           'lock_time',
                           'tx_index',
                           'double_spend',
                           'block_index',
                           'block_height',
                           'inputs',
                           'out'
                           ],
                           axis=1)

## Feature Engineering

As described in the README paper, there are a number of features that will provide meaninful representations of the problem domain to the machine learning model.

* size
* fee
* Time of transaction



In [ ]:
# fee percentage
merged_df['sat_per_byte'] = merged_df['fee']  / merged_df['size']

There is a discrepancy between two of the data sources...
The outdegree value for tx aff5627c3efd229dbf380d3f50a5600e45e6c0e154288814685dc90926145d23 from the original dataset (Kaggle) is 1, meaning there should only be a single output from this transaction. However, the vout_sz vlaue for the same row aquired from blockchain.com says the number of outputs is 2.

After looking at the outputs, the vout displays 2 outputs, yet the outdegree is 1.

There is another discrepancy with tx 25e3d5bf2334290b31c55129d100c2bce167a93985a35d6b8ebf44b6d04632ef where the same thing occurs. The Kaggle dataset reports there being 1 output, while the output string  in vout clearly shows 2.

 After [cross-referencing with other block explorers](https://blockexplorer.one/bitcoin/mainnet/tx/25e3d5bf2334290b31c55129d100c2bce167a93985a35d6b8ebf44b6d04632ef), it seems the indegree and outdegree from the Kaggle dataset can not be trusted, so I will instead use the vin_sz and vout_sz from the blockchain.com api data.

In [ ]:
# check for any nulls or nans
merged_df.isnull().sum()

in_btc          0
out_btc         0
total_btc       0
mean_in_btc     0
mean_out_btc    0
is_fraud        0
vin_sz          0
vout_sz         0
size            0
fee             0
time            0
sat_per_byte    0
dtype: int64

In [ ]:
merged_df.isna().sum()

in_btc          0
out_btc         0
total_btc       0
mean_in_btc     0
mean_out_btc    0
is_fraud        0
vin_sz          0
vout_sz         0
size            0
fee             0
time            0
sat_per_byte    0
dtype: int64

In [ ]:
merged_df

,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc,is_fraud,vin_sz,vout_sz,size,fee,time,sat_per_byte
0,50.000000,50.000000,100.000000,50.000000,50.000000,0,1,1,159,0,1277800165,0.000000
1,50.000000,50.000000,100.000000,50.000000,50.000000,0,1,1,224,0,1289311198,0.000000
2,2.700000,2.700000,5.400000,2.700000,1.350000,0,1,2,258,0,1271613446,0.000000
3,0.000000,50.000000,50.000000,0.000000,50.000000,0,1,1,134,0,1264625653,0.000000
4,0.000000,50.000000,50.000000,0.000000,50.000000,0,1,1,134,0,1262161248,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11320,23.224965,23.224865,46.449829,23.224965,11.612432,0,1,2,225,10000,1385735317,44.444444
11321,65.000000,64.990000,129.990000,32.500000,32.495000,1,2,2,439,1000000,1387540345,2277.904328
11322,2592.000000,2431.567900,5023.567900,2592.000000,2431.567900,1,1,2,257,1000000,1387540588,3891.050584
11323,100.000000,99.990000,199.990000,100.000000,49.995000,1,1,2,258,1000000,1387540345,3875.968992


# Normalising the Data
The data have a variety of ranges. To make learning easier, each feature needs to be normalised independently to have roughly the same range.

Therefore use Min-Max scaling


In [ ]:
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(merged_df), columns = merged_df.columns)

In [ ]:
scaled_data

,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc,is_fraud,vin_sz,vout_sz,size,fee,time,sat_per_byte
0,0.001037,0.001037,0.001037,0.001037,0.002075,0.0,0.000000,0.000000,0.000511,0.000000,0.246966,0.000000
1,0.001037,0.001037,0.001037,0.001037,0.002075,0.0,0.000000,0.000000,0.001188,0.000000,0.325150,0.000000
2,0.000056,0.000056,0.000056,0.000056,0.000056,0.0,0.000000,0.000361,0.001543,0.000000,0.204946,0.000000
3,0.000000,0.001037,0.000519,0.000000,0.002075,0.0,0.000000,0.000000,0.000250,0.000000,0.157484,0.000000
4,0.000000,0.001037,0.000519,0.000000,0.002075,0.0,0.000000,0.000000,0.000250,0.000000,0.140746,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11320,0.000482,0.000482,0.000482,0.000482,0.000482,0.0,0.000000,0.000361,0.001199,0.000103,0.980068,0.001142
11321,0.001349,0.001348,0.001349,0.000674,0.001348,1.0,0.001880,0.000361,0.003430,0.010309,0.992328,0.058542
11322,0.053782,0.050453,0.052117,0.053782,0.100906,1.0,0.000000,0.000361,0.001532,0.010309,0.992330,0.100000
11323,0.002075,0.002075,0.002075,0.002075,0.002075,1.0,0.000000,0.000361,0.001543,0.010309,0.992328,0.099612


In [ ]:
scaled_data.to_csv('/content/drive/MyDrive/final year/btc_tx_anomaly_detection/data/cleaned_undersampled_data.csv', index=False)